Colab Link: https://colab.research.google.com/drive/1VJwz2A1mdsRjSxZ9sihnilxI-0jBdXsM?usp=sharing

In [ ]:
import sklearn
import numpy as np

In [ ]:
import pandas as pd

# Loading data

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/FS-CSCI150-F21/FS-CSCI150-F21-Team4/main/datasets/train.csv')
data = data.drop(columns=['textID', 'selected_text'])
print(data)

                                                    text sentiment
0                    I`d have responded, if I were going   neutral
1          Sooo SAD I will miss you here in San Diego!!!  negative
2                              my boss is bullying me...  negative
3                         what interview! leave me alone  negative
4       Sons of ****, why couldn`t they put them on t...  negative
...                                                  ...       ...
27476   wish we could come see u on Denver  husband l...  negative
27477   I`ve wondered about rake to.  The client has ...  negative
27478   Yay good for both of you. Enjoy the break - y...  positive
27479                         But it was worth it  ****.  positive
27480     All this flirting going on - The ATG smiles...   neutral

[27481 rows x 2 columns]


In [ ]:
data.describe()

,text,sentiment
count,27480,27481
unique,27480,3
top,Its over for them,neutral
freq,1,11118


## Dropping any null values

In [ ]:
data = data.dropna()
#data['sentiment'] = data['sentiment'].replace(["negative","neutral", "positive"],[-1, 0, 1])
data.describe()

,text,sentiment
count,27480,27480
unique,27480,3
top,Its over for them,neutral
freq,1,11117


In [ ]:
training_data = data
print(training_data)

                                                    text sentiment
0                    I`d have responded, if I were going   neutral
1          Sooo SAD I will miss you here in San Diego!!!  negative
2                              my boss is bullying me...  negative
3                         what interview! leave me alone  negative
4       Sons of ****, why couldn`t they put them on t...  negative
...                                                  ...       ...
27476   wish we could come see u on Denver  husband l...  negative
27477   I`ve wondered about rake to.  The client has ...  negative
27478   Yay good for both of you. Enjoy the break - y...  positive
27479                         But it was worth it  ****.  positive
27480     All this flirting going on - The ATG smiles...   neutral

[27480 rows x 2 columns]


# Transforming text data into tfidf value

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.text)
X_train_counts.shape

(27480, 26439)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(27480, 26439)

# Fitting training data to Naive Bayes Model (temporary)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, training_data['sentiment'])

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
  ('vect', CountVectorizer()),
  ('tf', TfidfTransformer()),
  ('clf', MultinomialNB()),
])

## Using pipeline to transform data then train

In [ ]:
text_clf.fit(training_data.text, training_data.sentiment)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

# Mess around and put whatever text you wanna classify here

In [ ]:
test_case = ["What an amazing experience", "What are you doing today?", "okay man", "Wow that's great!", "really bad but okay i guess"]
X_new_counts = count_vect.transform(test_case)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
print(predicted)

['positive' 'neutral' 'neutral' 'positive' 'neutral']


## Pipeline test

In [ ]:
predicted = text_clf.predict(test_case)
print(predicted)

['positive' 'neutral' 'neutral' 'negative' 'neutral']


# Test set

In [ ]:
test_data = pd.read_csv('https://raw.githubusercontent.com/FS-CSCI150-F21/FS-CSCI150-F21-Team4/main/datasets/test.csv')
test_data = test_data.drop(columns=['textID'])
print(test_data)

                                                   text sentiment
0     Last session of the day  http://twitpic.com/67ezh   neutral
1      Shanghai is also really exciting (precisely -...  positive
2     Recession hit Veronique Branquinho, she has to...  negative
3                                           happy bday!  positive
4                http://twitpic.com/4w75p - I like it!!  positive
...                                                 ...       ...
3529  its at 3 am, im very tired but i can`t sleep  ...  negative
3530  All alone in this old house again.  Thanks for...  positive
3531   I know what you mean. My little dog is sinkin...  negative
3532  _sutra what is your next youtube video gonna b...  positive
3533   http://twitpic.com/4woj2 - omgssh  ang cute n...  positive

[3534 rows x 2 columns]


In [ ]:
predicted = text_clf.predict(test_data["text"])
np.mean(predicted == test_data.sentiment)

0.6247877758913413

oof not good enough

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('clf', SGDClassifier()),
])

text_clf.fit(training_data.text, training_data.sentiment)
predicted = text_clf.predict(test_data.text)
np.mean(predicted == test_data.sentiment)

0.7091114883984154

In [ ]:
from sklearn import metrics
print(metrics.classification_report(test_data.sentiment, predicted))

              precision    recall  f1-score   support

    negative       0.75      0.61      0.67      1001
     neutral       0.64      0.76      0.70      1430
    positive       0.80      0.72      0.76      1103

    accuracy                           0.71      3534
   macro avg       0.73      0.70      0.71      3534
weighted avg       0.72      0.71      0.71      3534



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_data.sentiment, predicted)

array([[ 614,  349,   38],
       [ 177, 1093,  160],
       [  32,  272,  799]])

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
  'vect__ngram_range': [(1, 1), (1, 2)],
  'tfidf__use_idf': (True, False),
  'clf__alpha': (1e-2, 1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [ ]:
 #gs_clf = gs_clf.fit(training_data.text, training_data.sentiment)

In [ ]:
#print(gs_clf.best_score_)
#for param_name in sorted(parameters.keys()):
  #print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [ ]:
from sklearn.linear_model import LogisticRegression
text_clf = Pipeline([
  ('vect', CountVectorizer()),
  ('tf', TfidfTransformer()),
  ('clf', LogisticRegression(max_iter=1000)),
])

text_clf.fit(training_data.text, training_data.sentiment)
predicted = text_clf.predict(test_data.text)
np.mean(predicted == test_data.sentiment)

0.7071307300509337

In [ ]:
from sklearn import metrics
print(metrics.classification_report(test_data.sentiment, predicted))

              precision    recall  f1-score   support

    negative       0.73      0.64      0.69      1001
     neutral       0.64      0.75      0.69      1430
    positive       0.80      0.71      0.75      1103

    accuracy                           0.71      3534
   macro avg       0.72      0.70      0.71      3534
weighted avg       0.72      0.71      0.71      3534

